$\textbf{LIBRARIES}$

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [31]:
def sep_print(): print('='*100)

In [32]:
path = "../Data/"

Quick look into data

In [33]:
df_states = pd.read_csv(path + 'stations_info.csv')
print(df_states.columns)

sep_print()
print(df_states.head(2))

Index(['file_name', 'state', 'city', 'agency', 'station_location',
       'start_month', 'start_month_num', 'start_year'],
      dtype='object')
  file_name           state        city agency          station_location  \
0     AP001  Andhra Pradesh    Tirupati  APPCB       Tirumala, Tirupati    
1     AP002  Andhra Pradesh  Vijayawada  APPCB  PWD Grounds, Vijayawada    

  start_month  start_month_num  start_year  
0        July                7        2016  
1         May                5        2017  


I have a lot of data, so we're going to choose only one state for analysis. 
I'm looking for the state that has the minimum number of agencies (each agency can have different measurements or columns name for the same feature, so choosing one will be easier) and the most data available and similar number of columns.

In [34]:
sum_dict = dict()
max_dict = dict()
min_dict = dict()

for i in os.listdir(path):

    if not i.endswith(".csv"):
        next

    prefix = i[:2]
    val = pd.read_csv(path + i, nrows= 1).shape[1]

    if prefix in sum_dict:
            sum_dict[prefix] += val
            max_dict[prefix] = val if max_dict[prefix] < val else  max_dict[prefix]
            min_dict[prefix] = val if min_dict[prefix] > val else  min_dict[prefix]
    else:
        sum_dict[prefix] = val
        max_dict[prefix] = val
        min_dict[prefix] = val

#Number of all columns in each state and max/ min number of column
smm_df = pd.DataFrame({'sum': sum_dict.values(), 'min': min_dict.values(), 'max': max_dict.values()}, index=sum_dict.keys()).sort_values(by= ["sum", "min", "max"])


In [35]:
#Number of diffrent agencies that measure air quility and how many .csv
df_states_sub = df_states[["file_name", "state", "agency"]]
df_states_sub["file_name"] = df_states_sub["file_name"].apply(lambda x: x[:2])

df_states_sub_info = df_states_sub[["file_name", "state", "agency"]].drop_duplicates().groupby(["file_name", "state"]).count()
df_states_sub_info = df_states_sub_info.join(
                pd.value_counts(df_states["state"]),
                on = "state"
            ).join(
                smm_df,
                on = "file_name"
            )

#Column rename
df_states_sub_info.columns = ['count_agency', 'count_csv', 'sum', 'min', 'max']

#Combining all created data
df_states_sub_info = df_states_sub_info.sort_values(['count_agency', 'count_csv', 'sum', 'min', 'max'], kind= 'heapsort')
print(df_states_sub_info[df_states_sub_info.count_agency == 1])

                             count_agency  count_csv  sum  min  max
file_name state                                                    
HP        Himachal Pradesh              1          1   23   23   23
JK        Jammu and Kashmir             1          1   23   23   23
MZ        Mizoram                       1          1   23   23   23
NL        Nagaland                      1          1   23   23   23
SK        Sikkim                        1          1   24   24   24
AR        Arunachal Pradesh             1          1   25   25   25
PY        Puducherry                    1          1   25   25   25
JH        Jharkhand                     1          2   29   10   19
TR        Tripura                       1          2   47   23   24
MN        Manipur                       1          2   48   24   24
ML        Meghalaya                     1          2   49   24   25
UK        Uttarakhand                   1          3   66   16   25
CH        Chandigarh                    1       

C:\Users\Ziewi\AppData\Local\Temp\ipykernel_46088\3135209361.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_states_sub["file_name"] = df_states_sub["file_name"].apply(lambda x: x[:2])


Looking at the output I chose Andhra Pradesh state. The reason is that this state have 10 .csv files within 23 to 25 column with data.
Rajasthan state could work too but my laptop wouldn't have been able to manage.

In [36]:
try:
    df_states.drop(columns= ["agency", "station_location", "start_month"], inplace= True)
except:
    pass
print(df_states.head(2))

  file_name           state        city  start_month_num  start_year
0     AP001  Andhra Pradesh    Tirupati                7        2016
1     AP002  Andhra Pradesh  Vijayawada                5        2017


In [37]:
df_states[df_states['state'] == 'Andhra Pradesh'].city.unique()

array(['Tirupati', 'Vijayawada', 'Visakhapatnam', 'Rajamahendravaram',
       'Amaravati', 'Anantapur', 'Chittoor', 'Kadapa'], dtype=object)

We are working with a large dataset which is split in multiple files. 
First I will create a function that will return a dataframe combining all datasheets measurements in a given state, so will see what data I'm working with.

In [38]:
def combine_state_file(state_name: str):
    file_name_short = df_states_sub[df_states_sub["state"] == state_name]["file_name"]

    global path
    combined_df = list()

    print(f"Combining a total of {len(file_name_short)} files")

    file_name_short = file_name_short.iloc[0]

    for file in  os.listdir(path):
        if file_name_short in file:
            print(file)

            file_data = pd.read_csv(path + file)
            file_data["city"] = df_states[df_states["file_name"] == file[:-4]]["city"].values[0]
            file_data["city"] = file_data["city"].astype("string")

            combined_df.append(file_data)

    return pd.concat(combined_df)

In [39]:
state_name = 'Andhra Pradesh'
df = combine_state_file(state_name)
df.info()

Combining a total of 10 files
AP001.csv
AP002.csv
AP003.csv
AP004.csv
AP005.csv
AP006.csv
AP007.csv
AP008.csv
AP009.csv
AP010.csv


I have limited memory and processing power, so I clear memory before going futher

In [40]:
l = %who_ls
exclude = ['df', 'np', 'os', 'pd', 'plt', 'sns', 'warnings' 'path', 'exclude', 'l']
for d in l:
    if d not in exclude:
        del globals()[d]

del l, exclude, d

In [41]:
%who_ls

['d', 'df', 'np', 'os', 'pd', 'plt', 'sns', 'val']

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 272217 entries, 0 to 1592
Data columns (total 29 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   From Date            272217 non-null  object 
 1   To Date              272217 non-null  object 
 2   PM2.5 (ug/m3)        215379 non-null  float64
 3   PM10 (ug/m3)         218510 non-null  float64
 4   NO (ug/m3)           220857 non-null  float64
 5   NO2 (ug/m3)          220797 non-null  float64
 6   NOx (ppb)            222419 non-null  float64
 7   NH3 (ug/m3)          216850 non-null  float64
 8   SO2 (ug/m3)          217459 non-null  float64
 9   CO (mg/m3)           219063 non-null  float64
 10  Ozone (ug/m3)        214914 non-null  float64
 11  Benzene (ug/m3)      221629 non-null  float64
 12  Toluene (ug/m3)      221602 non-null  float64
 13  Temp (degree C)      166170 non-null  float64
 14  RH (%)               223812 non-null  float64
 15  WS (m/s)             220

$\textbf{DATA PRCESSING}$

$\textbf{Exploratory Data Analysis (EDA)}$